# INFO-f422: ML Project

authors:
+ 1 
+ 2
+ 3

### Imports

In [31]:
import warnings
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

np.set_printoptions(threshold=3)

### Data loading

In [2]:
data_dir = "data"

X_g_train = np.load("../guided/guided_dataset_X.npy")
y_g_train = np.load("../guided/guided_dataset_y.npy")
X_g_test = np.load("../guided/guided_testset_X.npy")

X_f_train = np.load("../freemoves/freemoves_dataset_X.npy")
y_f_train = np.load("../freemoves/freemoves_dataset_y.npy")
X_f_test = np.load("../freemoves/freemoves_testset_X.npy")


In [3]:
print("Guided:")
print(f"X_g_train {X_g_train.shape} / y_g_train{y_g_train.shape} / X_g_test{X_g_test.shape}\n")
print("Free moves:")
print(f"X_f_train{X_f_train.shape} / y_f_train{y_f_train.shape} / X_f_test{X_f_test.shape}")

Guided:
X_g_train (5, 8, 230000) / y_g_train(5, 51, 230000) / X_g_test(5, 332, 8, 500)

Free moves:
X_f_train(5, 8, 270000) / y_f_train(5, 51, 270000) / X_f_test(5, 308, 8, 500)


### 1) Signal filtering

TODO: data exploration to take informed decision on filter (type of noise,....) to use and on filter parametres (no magic number)

In [4]:
from scipy.signal import butter, sosfiltfilt, firwin

In [5]:
nyq  = 1024 / 2
low  = 20  / nyq
high = 450 / nyq

sos = butter(4,[low,high], btype='band', output= 'sos')

for sess in range(X_g_train.shape[0]):
    for elec in range(X_g_train.shape[1]):
        # Application of the filtrage for x
        X_g_train[sess, elec, :] = sosfiltfilt(sos, X_g_train[sess, elec, :])

### 2) Dataset preparation

For this question, we decided to use the sliding_window_view function from the Numpy library for several reasons:

+ Fast vectorized numpy operations, compiled c-code (no python overhead, interpreter).

+ sliding_window_view function returns a view, no copy.

+ The function simplifies the implementation by automating window creation and indexing.

In [6]:
def create_overlap_windows(x, y, window_size, overlap, axis):

    step = int(window_size * (1 - overlap))

    # sliding_windows_view Generate all possible windows with the corresponding step, that not what we want.
    x_w = np.lib.stride_tricks.sliding_window_view(x,window_size,axis)
    y_w = np.lib.stride_tricks.sliding_window_view(y,window_size,axis)

    # only keep windows where the step is a multiple of our step 
    x_w = x_w[:,:,::step,:]
    y_w = y_w[:,:,::step,:]

    # We transpose the axes windows and electrode/signal 
    x_w = x_w.transpose(0, 2, 1, 3)     #  (session, window, electrode, time) and not  (session, electrode, window, time) TODO??
    y_w = y_w.transpose(0, 2, 1, 3)     # (session, window, signals, time)

    # Finaly, we keep only the last hand position (targets) for y, because for this project
    # we need to predict, for each window in x, the final hand position in the
    # same windows in the dataset y
    y_w = y_w[..., -1]  # (sessions, windows, targets)

    return x_w, y_w


X_g_train_wdw, y_g_train_wdw = create_overlap_windows(X_g_train, y_g_train, window_size=500, overlap=0.5, axis=2)
# !! windowed data is a view --> share original data memory (modify one, modify both)

print("Guided windowed:")
print(f"X_g_train_wdw {X_g_train_wdw.shape} / y_g_train_wdw{y_g_train_wdw.shape} / X_g_test{X_g_test.shape}")

Guided windowed:
X_g_train_wdw (5, 919, 8, 500) / y_g_train_wdw(5, 919, 51) / X_g_test(5, 332, 8, 500)


In [7]:
def quick_windows_tests(x, y):
    # (maybe automate tests given windowsize and overlap and consider internal frag (shoudl be discarded)
    
    x_w, y_w = create_overlap_windows(x, y, window_size=500, overlap=0.5, axis=2)    
    
    assert np.array_equal(x_w[0, 0, 0, :10], x[0, 0, :10]) # (sess 0) first 10 of electrode 0 in window 0
    assert np.array_equal(x_w[0, 1, 0, :10], x[0, 0, 250:260]) # (sess 0) first 10 of electrode 0 in window 1
    assert np.array_equal(x_w[0, 1, 4, :10], x[0, 4, 250:260]) # (sess 0) first 10 of electrode 4 in window 1
    assert np.array_equal(x_w[0, 918, 0, -10:], x[0, 0, 229990:230000]) # (sess 0) last 10 of electrode 0 in last window (918) - (perfect fit!)

quick_windows_tests(X_g_train, y_g_train)

#### 3) Cross validation strategy

For this question, we have thought about various methods of cross validation. First, our data are continous because it's a signal, so preserving temporal structure is important. We can’t use a method of cross validation which randomly shuffles our windows. 

We also need to prevents data leaking so we can't use a methode who use the windows of one session for training AND validation because we have overlapping data in each session, two windows in the same session can share the same datas, and if these two windows are in train and validation, it will lead to data leakage and overly optimistic performance (data in the train set will also be in the validation set). 

So it's naturally that we have chosen the "Leave One Group Out" method, this method will use each session as the validation set once and the other for training. We completly prevent data leakage because each session is indepandent from the other, and we reduce the bias because each session will be used for validation.

In our case, "LOGO" and "GroupKFold(5)" produce the same splits, but we choose "LOGO" because it's more explicit, readers will immediatly see that we use one session for validation each time while "GroupKFold" need to have 5 in parameter to do the same thong

In [8]:
x_shape = X_g_train_wdw.shape
y_shape = y_g_train_wdw.shape

groups = np.repeat(np.arange(1,x_shape[0]+1),x_shape[1] ) # 111 (919 times), 222 (919 times), ...
print(f"groups{groups.shape}\n")

# We need to flatten the dataset x and y because the function logo (and latter "croos_val_score")
# want all the data in a 2d list, we will know have  the dataset X for exemple.
# [4595, 4000] and not [5,919,8,500], 4595 is the multiplication of 5 and 919 (3500 = 8*500), and y 
# [4595,51] and not [5,919,51].
# Now all the windows are store in a list and the "groups" list above allow the function 
# logo to know at wich session each windows belong
# The windows 3 for example (x_windows_flat[2]) belong to the sessions groups[2] = 1
X_g_train_wdw_flat = X_g_train_wdw.reshape(x_shape[0] * x_shape[1], x_shape[2] * x_shape[3])
y_g_train_wdw_flat = y_g_train_wdw.reshape(y_shape[0] * y_shape[1], y_shape[2])

print("Guided windowed flattened:")
print(f"X_g_train_wdw_flat{X_g_train_wdw_flat.shape} / y_g_train_wdw_flat{y_g_train_wdw_flat.shape}")

groups(4595,)

Guided windowed flattened:
X_g_train_wdw_flat(4595, 4000) / y_g_train_wdw_flat(4595, 51)


In [9]:
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.linear_model import Ridge, Lasso, ElasticNet

In [10]:
# %%time

# np.random.seed(0)

logo = LeaveOneGroupOut()

lasso_model = Lasso(max_iter=10) # the futur model 
ridge_model = Ridge()

rmse_scorer = make_scorer(
    lambda y_true, y_pred: np.sqrt(mean_squared_error(y_true, y_pred)),
    greater_is_better=False  # Score near 0 is better 
)

def cross_validation_with_scores(X,Y,groups,model,cv,scoring):
    # The cross_val_score function by sklearn will execute our cv and return a tab 
    neg_rmse_scores = cross_val_score(
        model,
        X,
        Y,
        groups=groups,
        cv=logo,
        scoring=rmse_scorer,
        n_jobs=-1 # Use all cores 
    )
    
    # Conversion of negatifs scores into positifs (convention of sklearn)
    rmse_scores = -neg_rmse_scores  
    print("RMSE for each folder:", rmse_scores)

cross_validation_with_scores(X_g_train_wdw_flat,y_g_train_wdw_flat,groups,lasso_model,logo,rmse_scorer)

/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.965e+03, tolerance: 4.492e+00
  model = cd_fast.enet_coordinate_descent(
/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.652e+03, tolerance: 4.499e+00
  model = cd_fast.enet_coordinate_descent(
/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisati

RMSE for each folder: [19.80889373 20.70289974 23.42573968 20.45592827 21.90544978]


/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.782e+05, tolerance: 1.351e+02
  model = cd_fast.enet_coordinate_descent(


In [14]:
# rmse context
sess = 0
y_max = np.max(y_g_train_wdw[sess])
y_min = np.min(y_g_train_wdw[sess])
y_mean = np.mean(y_g_train_wdw[sess])

print(f"Session {sess} target info:\n  min = {y_min}\n  max = {y_max}\n  mean = {y_mean}")

Session 0 target info:
  min = -108.68231864942676
  max = 44.76897408739836
  mean = -5.73247691191569


In [15]:

for i, (train_index, test_index) in enumerate(logo.split(X_g_train_wdw_flat, y_g_train_wdw_flat, groups)):
    print(f"Fold {i}")
    print(f"   train groups: {np.unique(groups[train_index])}")
    print(f"   test groups: {np.unique(groups[test_index])}")

Fold 0
   train groups: [2 3 4 5]
   test groups: [1]
Fold 1
   train groups: [1 3 4 5]
   test groups: [2]
Fold 2
   train groups: [1 2 4 5]
   test groups: [3]
Fold 3
   train groups: [1 2 3 5]
   test groups: [4]
Fold 4
   train groups: [1 2 3 4]
   test groups: [5]


#### Question 5

In [16]:
#pip install pyriemann
from pyriemann.estimation import Covariances
from pyriemann.tangentspace import TangentSpace


In [17]:
# Covariances method

# First we need to reshape the dataset X, from a 2d tab to a 3d tab, 
X_g_train_reshape = X_g_train_wdw_flat.reshape(4595,8,500)

# We initalise an object of the class Covariances from the PyRiemann lib and call the fit_transform 
# method for having a tab of covrainces matrices of each windows 
covariance = Covariances(estimator='oas')
SPD_tab = covariance.fit_transform(X_g_train_reshape) 
# print(SPD_tab.shape) # (4595,8,8)

# The tangentSpace class will calcul the log map of each SPD matrices. This projection will transform our SPD matrices into euclidean vector 
ts = TangentSpace()
tangent_tab = ts.fit_transform(SPD_tab)
#print(tangent_tab.shape) # (4595,36) Now that we have a 2d tab, we can use traditional regression algorithms


# Now we juste need to use the cv function we build above 
cross_validation_with_scores(tangent_tab,y_g_train_wdw_flat,groups,lasso_model,logo,rmse_scorer)

/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.001e+03, tolerance: 4.037e+01
  model = cd_fast.enet_coordinate_descent(
/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.576e+02, tolerance: 5.886e+01
  model = cd_fast.enet_coordinate_descent(
/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisati

RMSE for each folder: [8.77911528 8.27891509 8.21374781 7.32334406 8.3483254 ]


In [18]:
# pip install -U skorch
import torch
import torch.nn as nn
from skorch import NeuralNetRegressor


In [ ]:
## NN method


class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack  = nn.Sequential(
            nn.Linear(4000, 512), # 4000 datas in one windows
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 51)
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


# We need to convert the dataset type from float64 to float32 because pyTorch nn.Linear  are in float32
# This will not affect our data because 32 bits is stil massive and our emg signal don't take that much space
print(X_g_train_wdw_flat.dtype)   
print(y_g_train_wdw_flat.dtype)   

x = X_g_train_wdw_flat.astype('float32')
y = y_g_train_wdw_flat.astype('float32')


net = NeuralNetRegressor(
    module=NeuralNetwork,                 
    max_epochs=20,                 
    lr=1e-3,                       
    batch_size=64,
    optimizer=torch.optim.Adam,   
)


cross_validation_with_scores(x,y,groups,net,logo,rmse_scorer)


float64
float64


/home/khalil/ls/envs/sfml/lib/python3.12/site-packages/skorch/net.py:2261: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  cuda_attrs = torch.load(f, **load_kwargs)
/home/khal

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1      324.8340      281.7868  2.7386
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1      321.7629      281.2231  2.7120
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1      328.3908      276.0523  2.7123
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1      322.4828      279.6053  2.7067
  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1      329.6119      283.0252  2.7889
      2      245.0988      290.8188  2.6445
      2      247.6856      265.8422  2.6336
      2      250.2326      286.3130  2.7014
      2      253.2321      287.7198  2.6875
      2      266.0030      289.6881  2.6679
      3      148.1459      296.9657  2.6300
      3      140.0952      273.5769  2.6294
      3      152.2091      279.6